In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "smartcity.settings")
import django
django.setup()

In [3]:
from app import models

In [4]:
employees = models.Employee.objects.all()

In [5]:
user = models.User.objects.get(username='EMM18316')
emp = models.Employee.objects.get(user_id=user.id)
mycases = models.CaseManager.objects.filter(responder_id=emp.id)

DoesNotExist: User matching query does not exist.

In [9]:
mycases.filter(status='closed')

<QuerySet [<CaseManager: CaseManager object>, <CaseManager: CaseManager object>]>

In [11]:
models.CaseManager.objects.count()

2749

# Generate Vacancy data

In [73]:
from collections import Counter
from difflib import SequenceMatcher
import datetime

In [10]:
employees.count()

1326

In [30]:
faults = models.Fault.objects.all()

In [ ]:
Counter(flist)

In [31]:
flist = [f.category for f in faults]

In [34]:
categories = list(pd.unique(flist))

In [35]:
categories

[u'Water', u'Waste', u'Sewage', u'Electric', u'Traffic', u'Street']

In [25]:
import pickle

In [28]:
skills = pickle.load(open('data/skills.pkl'))

In [51]:
wanted_skills = []

In [53]:
for skill in skills:
    for cat in categories:
        if SequenceMatcher(None, cat, skill).ratio() > .4:
            wanted_skills.append((skill, cat))
#            print("{}; {}".format(skill, cat))

In [56]:
degree = ['BSc', 'BA', 'BTech', 'Diploma', 'National Certificate']

In [17]:
jds = employees.values_list('job_desc')

In [20]:
jtitle = employees.values_list('job_title')

In [70]:
jtitle

<QuerySet [(u'Electrical Engineer',), (u'Electrical Engineer',), (u'Civil Engineer',), (u'Civil Engineer',), (u'Civil Engineer',), (u'Electrical Engineer',), (u'Plumber',), (u'Civil Engineer',), (u'Plumber',), (u'Civil Engineer',), (u'Plumber',), (u'Plumber',), (u'Civil Engineer',), (u'Electrical Engineer',), (u'Civil Engineer',), (u'Plumber',), (u'Electrical Engineer',), (u'Plumber',), (u'Plumber',), (u'Electrical Engineer',), '...(remaining elements truncated)...']>

In [83]:
models.Vacancy.objects.all().delete()

(663, {u'app.Application': 0, u'app.Vacancy': 663})

In [84]:
for i in range(int(len(employees) - len(employees) * .5)):
    title = jtitle[np.random.randint(0, len(jtitle))]
    jd = jds[np.random.randint(0, len(jds))]
    cat = wanted_skills[np.random.randint(0, len(wanted_skills))][1]
    skill = pd.unique([wskills for wskills in wanted_skills if cat == wskills[1]][:np.random.randint(1,6)])[0]
    qualifications = str(degree[np.random.randint(0, len(degree))]) + '., ' + title[0]
    posting = datetime.datetime.now().date() - datetime.timedelta(np.random.randint(0, 11))
    closing = posting + datetime.timedelta(np.random.randint(2, 21))
    
    models.Vacancy.objects.create(
        title = title[0],
        description = jd[0],
        skills = skill,
        qualifications = qualifications,
        posting_date =  posting,
        closing_date = closing)

In [80]:
datetime.datetime.now().date() - datetime.timedelta(1)

datetime.date(2017, 11, 3)

In [82]:
models.Vacancy.objects.count()

663

# Generate Cases

In [ ]:
faults = models.Fault.objects.all()

In [ ]:
faults.count()

In [ ]:
emps = models.Employee.objects.all()

In [ ]:
status = ['closed', 'open', 'pending']

In [ ]:
reason = ['no resources', 'high risk', 'unable to verify']

In [ ]:
len(models.Employee.objects.filter(specializations__contains='sewage'))

In [ ]:
employees = models.Employee.objects.all()

In [ ]:
flts = [fault.category for fault in faults]

In [ ]:
from collections import Counter

In [ ]:
Counter(flts)

In [ ]:
pd.unique(flts)

In [ ]:
models.CaseManager.objects.all().delete()

for fault in faults:
    stat = status[np.random.randint(0, 3)]
    reas = ('', reason[np.random.randint(0, 3)])[stat == 'pending']
    try:
        employees = models.Employee.objects.filter(specializations__contains=fault.category)
        employee = employees[np.random.randint(0, len(employees))]
        models.CaseManager.objects.create(
        fault = fault,
        responder = employee,
        status = stat,
        reason = reas)
    except:
        pass

In [ ]:
models.CaseManager.objects.all().count()
    

In [ ]:
models.CaseManager.objects.count()

In [ ]:
!ls -l ../data/

In [ ]:
data = pd.read_excel("../data/emm_queries.xlsx", sheetname=None)

In [ ]:
water_df = pd.DataFrame(raw_data['Water problems'])
waste_df = pd.DataFrame(raw_data[ 'Waste problems'])
sewage_df = pd.DataFrame(raw_data['Sewerage problems'])
electric_df = pd.DataFrame(raw_data['Electricity problems'])
traffic_df = pd.DataFrame(raw_data['Traffic lights problems'])
street_df = pd.DataFrame(raw_data['Street Lighting Fault'])
road_df = pd.DataFrame(raw_data['Road related problems'])

In [ ]:
#dfs = ['water_df', 'waste_df', 'sewage_df', 'electric_df', 'traffic_df', 'street_df', 'road_df']
dfs = [water_df, waste_df, sewage_df, electric_df, traffic_df, street_df, road_df]

In [ ]:
waste_df.head()

In [ ]:
count = 1
water_defects = ['leaking tap', 'burst pipe', 'cutoff', 'shortages']
sewage_defects = ['leaking toilet', 'burst pipe', 'blocked']
electric_defects = ['power outage', 'mulfunctions', 'no electricity']

In [ ]:
def change_defect_col_bame(df):
    return df.rename(columns={'defect_type':'DEFECT'}, inplace=True)

In [ ]:
electric_df.head()

In [ ]:
water_df.head()

In [ ]:
waste_df.head()

In [ ]:
for df in dfs:
    df.dropna(axis=1, how='all', inplace=True)
    df.drop_duplicates(inplace=True)
    
    if count == 1:
        df['CATEGORY'] = 'WATER'
        df['DEFECT'] = None
        df.DEFECT = df.DEFECT.transform(lambda x: str(x).replace('None', water_defects[np.random.randint(0, 4)]))
    elif count == 2:
        df['CATEGORY'] = 'WASTE'
        df = change_defect_col_bame(df)
    elif count == 3:
        df['CATEGORY'] = 'SEWAGE'
        df['DEFECT'] = None
        df.DEFECT = df.DEFECT.transform(lambda x: str(x).replace('None', sewage_defects[np.random.randint(0, 3)]))
    elif count == 4:
        df['CATEGORY'] = 'ELECTRIC'
        df['DEFECT'] = None
        df.DEFECT = df.DEFECT.transform(lambda x: str(x).replace('None', electric_defects[np.random.randint(0, 3)]))
    elif count == 5:
        df['CATEGORY'] = 'TRAFFIC'
#        df.rename({'defect_type':'DEFECT'}, inplace=True)
        df = change_defect_col_bame(df)
    elif count == 6:
        df['CATEGORY'] = 'STREET'
#        df.rename({'defect_type':'DEFECT'}, inplace=True)
        df = change_defect_col_bame(df)
    else:
        df['CATEGORY'] = 'ROAD'
#        df.rename({'defect_type':'DEFECT'}, inplace=True)
        df = change_defect_col_bame(df)
    
    count += 1

In [ ]:
street_df.head()

In [ ]:
xls_writer = pd.ExcelWriter("../data/emm_quies_proc.xlsx", engine='xlsxwriter')

In [ ]:
count = 0
for df in dfs:
    df.to_excel(xls_writer, sheet_name=str(count), index=False, header=True)
    count += 1

In [ ]:
xls_writer.save()

In [ ]:
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.plotting import figure

In [ ]:
output_file("visuals.html")

In [ ]:
street_df.head()

In [ ]:
pd.unique(street_df.DEFECT)

In [ ]:
street_df['month_submitted'] = street_df.DateSubmitted.transform(lambda x: str(x).split('-')[1])

In [ ]:
p = figure(x_range=pd.unique(list(street_df.month_submitted)), 
           plot_height=350, 
           title="Defect Type by Month", 
           toolbar_location=None, 
           tools="")

p.vbar_stack(list(pd.unique(street_df.DEFECT)), 
             x='fruits', 
             width=0.9, 
             source=street_df, legend=[value(x) for x in list(pd.unique(street_df.DEFECT))])

In [ ]:
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"